### [Keras API](https://keras.io/api/)
### [Numpy API](https://numpy.org/doc/stable/reference/index.html)

自定義 Layer
自定義 Model  
模型儲存  
檔案讀取  

model.save_weigits(filepath)  
model = create_model()  
model.load_weights(filepath)  

model.save(filepath)  
model = models.load_model(filepath)  

numpy.random.randint(low, high=None, size=None, dtype='l')  

tf.data.TextLineDataset(filenames)  
tf.keras.utils.get_file(fname, origin=URL)  
pd.read_csv(filepath)  

In [3]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers, models

seed = 13
np.random.seed(seed)

In [90]:
seed = 13
np.random.seed(seed)
num_words = 100
txt_source_1 = np.random.randint(low=50, size=10)
txt_source_2 = np.random.randint(low=50,high=100, size=10)
embedding_1 = layers.Embedding(input_dim=num_words, output_dim=13)
embedding_2 = layers.Embedding(input_dim=num_words, output_dim=13)
lstm_1 = layers.LSTM(32)
lstm_2 = layers.LSTM(64)

output_layer_1 = layers.Dense(units=1)
output_layer_2 = layers.Dense(units=100)

print("txt_source_1",txt_source_1)
print("txt_source_2",txt_source_2)
print("txt_source_1",txt_source_1.shape,"\t","txt_source_2",txt_source_2.shape)
print("inputs 的第0維是不確定尺寸的batch_size，layers 通常會忽視第0維，所以第0維要先 expand_dims 擴充出來")
txt_source_1 =tf.expand_dims(txt_source_1, axis=0)
txt_source_2 =tf.expand_dims(txt_source_2, axis=0)
print("txt_source_1",txt_source_1.shape,"\t","txt_source_2",txt_source_2.shape)
print("embedding_1 input_dim=100 , output_dim=13, inputs=(None, 10)")
print("embedding_1 outputs.shape",embedding_1.compute_output_shape(input_shape=(None,10)))
print("embedding_2 input_dim=100 , output_dim=13, inputs=(None, 10)")
print("embedding_2 outputs.shape",embedding_2.compute_output_shape(input_shape=(None,10)))
print("lstm_1 units=32, inputs=(None, 10, 13)")
print("lstm_1 outputs.shape",lstm_1.compute_output_shape(input_shape=(None, 10, 13)))
print("lstm_2 units=64, inputs=(None, 10, 13)")
print("lstm_2 outputs.shape",lstm_2.compute_output_shape(input_shape=(None,10,13)))
print()
print("第一批次投入, batch_size=1")
lstm_1_outputs = lstm_1(embedding_1(txt_source_1))
lstm_2_outputs = lstm_2(embedding_2(txt_source_2))
print("lstm_1_outputs.shape",lstm_1_outputs.shape)
print("lstm_2_outputs.shape",lstm_2_outputs.shape)
concate_layer_outputs = layers.concatenate([lstm_1_outputs,lstm_2_outputs])
print("合併 lstm_1_outputs lstm_2_outputs ，第0軸同值，以第0軸進行合併")
print("concate_layer_outputs.shape",concate_layer_outputs.shape)
print()
print("合併層分別餵值給 不同的 predict 層，給出不同的 predict")

predict_1 = output_layer_1(concate_layer_outputs) 
predict_2 = output_layer_2(concate_layer_outputs) 
print("predict_1.shape",predict_1.shape)
print("predict_2.shape",predict_2.shape)

txt_source_1 [18 48 10 16 38 34 25 44 35 26]
txt_source_2 [52 96 87 63 60 59 61 52 82 69]
txt_source_1 (10,) 	 txt_source_2 (10,)
inputs 的第0維是不確定尺寸的batch_size，layers 通常會忽視第0維，所以第0維要先 expand_dims 擴充出來
txt_source_1 (1, 10) 	 txt_source_2 (1, 10)
embedding_1 input_dim=100 , output_dim=13, inputs=(None, 10)
embedding_1 outputs.shape (None, 10, 13)
embedding_2 input_dim=100 , output_dim=13, inputs=(None, 10)
embedding_2 outputs.shape (None, 10, 13)
lstm_1 units=32, inputs=(None, 10, 13)
lstm_1 outputs.shape (None, 32)
lstm_2 units=64, inputs=(None, 10, 13)
lstm_2 outputs.shape (None, 64)

第一批次投入, batch_size=1
lstm_1_outputs.shape (1, 32)
lstm_2_outputs.shape (1, 64)
合併 lstm_1_outputs lstm_2_outputs ，第0軸同值，以第0軸進行合併
concate_layer_outputs.shape (1, 96)

合併層分別餵值給 不同的 predict 層，給出不同的 predict
predict_1.shape (1, 1)
predict_2.shape (1, 100)


### 自定義 Layer

In [140]:
class MyDenseLayer(layers.Layer):
    def __init__(self, units):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = units       
        
    def build(self, input_shape):
        self.kernel = self.add_variable(name="kernel",
                      shape=(int(input_shape[-1]), self.num_outputs))
        self.op_shape = (int(input_shape[0]), self.num_outputs)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

shape = (5,10)
layer = MyDenseLayer(13)
layer.build(shape)
print("input_shape",shape)
print("input_shape[-1] =",shape[-1])
print("MyDenseLayer(units=13) 自定義內核kernel.shape是")
print("(input_shape[-1],units)\t", "self.kernel",layer.kernel.shape)

np.random.seed(seed)
inputs = np.random.normal(size=shape).astype(np.float32)
outputs = layer(inputs)

print("call 方法定義 outputs 是 inputs 與 kernel 的乘積")
print("inputs(5,10) * kernel(10,13) = (5,13)")
print("outputs.shape",layer(inputs).shape)
print("outputs")
print(outputs)

input_shape (5, 10)
input_shape[-1] = 10
MyDenseLayer(units=13) 自定義內核kernel.shape是
(input_shape[-1],units)	 self.kernel (10, 13)
call 方法定義 outputs 是 inputs 與 kernel 的乘積
inputs(5,10) * kernel(10,13) = (5,13)
outputs.shape (5, 13)
outputs
tf.Tensor(
[[-1.2395321   0.5037262  -0.3948251   0.02081331  0.56775415 -0.2135733
  -0.39356157  0.25855166  0.58117986 -0.3557209  -0.40026766 -0.8912854
   1.1180192 ]
 [-1.4274954   1.0056986  -0.7326327  -0.7313405  -0.63006926 -0.9528735
   0.6477816  -0.19287156 -0.41200137 -0.7933595  -1.5083256  -0.08673576
  -0.32435635]
 [ 1.0695055  -1.5059757  -0.27395836 -1.5259657   0.98028755 -0.16290466
  -1.5180768  -0.38570347  0.5167788   0.37858826  2.3674963  -0.7480683
   1.184243  ]
 [ 1.1673536  -0.99624455  0.16720666 -0.3855187   0.03444865  1.0790647
  -1.1444905   0.5952683   0.5254709   0.97738564  1.3313545  -0.74869365
   1.0607816 ]
 [ 0.50073826  0.16083673 -0.60955906 -1.4797384   0.12872718  1.0826287
  -0.9371912  -0.01380042 -0.018

### 自定義 Model

In [147]:
class MyModel(models.Model):
    def __init__(self):
        super(MyModel, self).__init__()        
        self.layer_1 = MyDenseLayer(5)
        self.layer_2 = MyDenseLayer(10)
        
    def call(self, inputs):     
        x = self.layer_1(inputs)
        x = self.layer_2(x)
        return x
    
shape = (3, 4)
model = MyModel()
model.build(input_shape=shape)
print("input_shape=",shape)
print("layer_1 units = 5, outputs=",model.layers[0].op_shape)
print("layer_2 units = 10, outputs=",model.layers[1].op_shape)
inputs = np.random.normal(size=shape).astype(np.float32)
predict = model(inputs)
print("model.predict.shape",predict.shape)
model.summary()
print("predict")
print(predict)

input_shape= (3, 4)
layer_1 units = 5, outputs= (3, 5)
layer_2 units = 10, outputs= (3, 10)
model.predict.shape (3, 10)
Model: "my_model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense_layer_55 (MyDenseLa multiple                  20        
_________________________________________________________________
my_dense_layer_56 (MyDenseLa multiple                  50        
Total params: 70
Trainable params: 70
Non-trainable params: 0
_________________________________________________________________
predict
tf.Tensor(
[[ 0.80190194 -1.9273742   0.24396832  0.14072034 -1.0808374   0.8717728
   0.5275319   0.03014566  0.6637947  -0.14115961]
 [ 0.07960504 -0.9355753  -0.28438684  0.3910032  -0.46222162  0.19415335
   0.14213584 -0.04647347  0.4220163  -0.07509527]
 [-0.23284486  1.1881171  -0.65556085  0.13655096 -0.02338164 -0.5181538
  -0.06247193  0.17872141 -0.6896861  -0.23201069]], shape=(

### 模型儲存

In [150]:
filename = "weights.ckpt"
filename2= "model.h5"
model.save_weigits(filename)
model.save(filename2)

model = create_model()
model.load_weights(filename)

model = models.load_model(filename2)

In [151]:
import pandas as pd
import tensorflow as tf
from tensorflow import data
import time

filepath = "GOOG_Stock_Price_Train.csv"

In [152]:
%time tdf = tf.data.TextLineDataset(filepath)

Wall time: 143 ms


In [153]:
head =  tdf.take(5)
for i, item in enumerate(head):    
    print(item)

tf.Tensor(b'Date,Open,High,Low,Close,Adj Close,Volume', shape=(), dtype=string)
tf.Tensor(b'2012-01-03,324.360352,331.916199,324.077179,330.555054,330.555054,7400800', shape=(), dtype=string)
tf.Tensor(b'2012-01-04,330.366272,332.959412,328.175537,331.980774,331.980774,5765200', shape=(), dtype=string)
tf.Tensor(b'2012-01-05,328.925659,329.839722,325.994720,327.375732,327.375732,6608400', shape=(), dtype=string)
tf.Tensor(b'2012-01-06,327.445282,327.867523,322.795532,322.909790,322.909790,5420700', shape=(), dtype=string)


In [154]:
%time pd_df= pd.read_csv(filepath)

Wall time: 69 ms


In [155]:
pd_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2012-01-03,324.360352,331.916199,324.077179,330.555054,330.555054,7400800
1,2012-01-04,330.366272,332.959412,328.175537,331.980774,331.980774,5765200
2,2012-01-05,328.925659,329.839722,325.994720,327.375732,327.375732,6608400
3,2012-01-06,327.445282,327.867523,322.795532,322.909790,322.909790,5420700
4,2012-01-09,321.161163,321.409546,308.607819,309.218842,309.218842,11720900


In [156]:
pd_df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1.258000e+03
mean,532.895561,537.059719,528.200106,532.719408,532.719408,3.164147e+06
std,152.229138,153.337676,150.874621,152.074831,152.074831,2.280797e+06
min,278.359528,280.436035,276.461884,277.718719,277.718719,7.900000e+03
25%,403.005081,405.652870,400.665306,403.016274,403.016274,1.624850e+06
50%,536.841492,540.345002,532.517273,536.695007,536.695007,2.573150e+06
75%,654.922501,662.587494,644.798752,652.172485,652.172485,4.133800e+06
max,816.679993,816.679993,805.140015,813.109985,813.109985,2.504650e+07


In [157]:
pd_df.describe(include="all")

,Date,Open,High,Low,Close,Adj Close,Volume
count,1258,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1.258000e+03
unique,1258,NaN,NaN,NaN,NaN,NaN,NaN
top,2012-08-15,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,532.895561,537.059719,528.200106,532.719408,532.719408,3.164147e+06
std,NaN,152.229138,153.337676,150.874621,152.074831,152.074831,2.280797e+06
min,NaN,278.359528,280.436035,276.461884,277.718719,277.718719,7.900000e+03
25%,NaN,403.005081,405.652870,400.665306,403.016274,403.016274,1.624850e+06
50%,NaN,536.841492,540.345002,532.517273,536.695007,536.695007,2.573150e+06
75%,NaN,654.922501,662.587494,644.798752,652.172485,652.172485,4.133800e+06
